In [9]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s - %(message)s')

#import experiments
import SeqDHBM
import fasta
import pdbfiles
import sys, os
import shutil

def workflow(jobnum= 0, fastafile=None, pdbs=[], pdbid="", mode=True):
    """ Runs the workflow 
    
    Params:
    - fasta: One fasta file
    - pdbfiles: list of pdb filenames, they should be in the root dir
    - pdbid: string with PDB codes (comma separated)
    - knowstruc: boolean: we use services to obtain the structure if False"""
    # TODO: use jobnum to organize submissions
    # Deal with the fasta file with multiple sequences
    if fastafile:
        seq_dict = fasta.fasta_to_seq(fastafile)
        fasta.organize_sequences(seq_dict)
        print("fasta:")
        print(seq_dict)

    # Deal with the pdb ids (by downloading them)
    pdbid_dict = {} # stores the pdbfile path to each pdb id
    if pdbid:
        l = pdbfiles.text_to_list(pdbid)
        pdbid_dict = pdbfiles.get_pdb_files(l)
        print("pdbid")
        print(pdbid_dict)
    
    # Arrange the pdb files (user-submitted or downloaded)
    ## Put the structure file in folders - for docking and MD
    ## get the primary sequence for motif search
    for file in pdbs:
        try:
            folder = ".".join(file.split(".")[:-1]) # remove extension
        except:
            folder = file
        try:
            os.makedirs(folder, exist_ok=True)
            shutil.move(file, folder+"/"+file)
            pdbid_dict[folder] = folder+"/"+file
        except:
            logging.error("Could not move file"+ file + "\nSkiping it!") # TODO: List of warnings and errors
            # TODO: organize the files into folders
    
    # TODO: convert the pdbs into fasta
    for file in pdbid_dict.values():
        seq = pdbfiles.structure_to_fasta(file)
        if seq:
            seq_dict[file] = (">header", seq)
    # Run the motif check
    print("*"*30)
    print(seq_dict)
    
    spacerinfo = {}
    
    for folder, (header, seq) in seq_dict.items():
        """fo = "Saved_run/IL-1A(271)/IL-1A(271).fasta"
    
        SeqDHBM.ReadFasta(fo)"""
        #print(folder, "$", seq)
        coordsites = SeqDHBM.SpotCoordinationSite({folder: seq}, mode)
        for folder, coordsite in coordsites.items():
            # TODO: Handle what is to be done with the coordination sites
            # Output? Save in our user output?
            print(folder)
            print(coordsite)

if (__name__ == "__main__"):

    fastafile = '/home/imhof_team/Public/mauricio/workflow/test.fasta'
    pdbids = "1S0L,1L2H,1I1B,1HIB,1I8H"
    pdbs=[]
    mode = "structure"
    
    for params in sys.argv:
        if params.startswith("-fasta="):
            fastafile = params[7:]
    
    for params in sys.argv:
        if params.startswith("-pdbid="):
            pdbids = params[7:]

    for params in sys.argv:
        if params.startswith("-pdbfiles="):
            # param pdbid must be formatted as list
            pdbs = eval(params[10:])
            
    for params in sys.argv:
        if params.startswith("-mode="):
            # param pdbid must be formatted as list
            mode = params[6:].lower()
    
    assert mode in ["structure", "wesa"], ("Please choose one of the modes: \n"
    "- structure: if structure is known (default)\n"
    "- wesa: to use our services to PREDICT surface accessibility\n")
            
    workflow(jobnum=0, fastafile=fastafile, pdbs=pdbs, pdbid=pdbids, mode=mode)



fasta:
{'271(271)': ('>IL-1A|271', 'MAKVPDMFEDLKNCYSENEEDSSSIDHLSLNQKSFYHVSYGPLHEGCMDQSVSLSISETSKTSKLTFKESMVVVATNGKVLKKRRLSLSQSITDDDLEAIANDSEEEIIKPRSAPFSFLSNVKYNFMRIIKYEFILNDALNQSIIRANDQYLTAAALHNLDEAVKFDMGAYKSSKDDAKITVILRISKTQLYVTAQDEDQPVLLKEMPEIPKTITGSETNLLFFWETHGTKNYFTSVAHPNLFIATKQDYWVCLAGGPPSITDFQILENQA'), '268(300)': ('>IL-1B|268', 'MAEVPELASEMMAYYSGNEDDLFFEADGPKQMKCSFQDDLCPLDGGIQLRISDHHYSKGFRQAASVVVAMDKLRKMLVPCPQTFQENDLSTFFPFIFEEEPIFFDTWDNEAYVHDAPVRSLNCTLRDSQQKSLVMSGPYELKALHLQGQDMEQQVVFSMSFVQGEESNDKIPVALGLKEKNLYLSCVLKDDKPTLQLESVDPKNYPKKKMEKRFVFNKIEINNKLEFESAQFPNWYISTSQAENMPVFLGGTKGGQDITDFTMQFVSSAQNQWLQDMTTHLILRSFKEFLQSSLRALRQM')}
Structure exists: '1S0L/pdb1s0l.ent' 
Structure exists: '1L2H/pdb1l2h.ent' 
Structure exists: '1I1B/pdb1i1b.ent' 
Structure exists: '1HIB/pdb1hib.ent' 
Structure exists: '1I8H/pdb1i8h.ent' 
pdbid
{'1S0L': '1S0L/pdb1s0l.ent', '1L2H': '1L2H/pdb1l2h.ent', '1I1B': '1I1B/pdb1i1b.ent', '1HIB': '1HIB/pdb1hib.ent', '1I8H': '1I8H/pdb1i8h.ent'}
********************

In [11]:
import logging
logging.debug('This is a log message.'+str(1))

DEBUG - This is a log message.1
